# 线性回归

训练一个模型指的是设置模型的参数使得这个模型在训练集的表现较好。为此，首先需要找到一个衡量模型好坏的评定方法。之前，介绍到在回归模型上，__最常见的评定标准是均方根误差（RMSE）__。因此，为了训练一个线性回归模型，你需要找到一个 $\theta$ 值，它使得均方根误差（标准误差）达到最小值。实践过程中，最小化均方误差比最小化均方根误差更加的简单，这两个过程会得到相同的 $\theta$ ，因为函数在最小值时候的自变量，同样能使函数的方根运算得到最小值。

## 正规方程



特征数量很多时，求解速度特别慢。

## 梯度下降

### 批量梯度下降

### SGD

### mini-batch 梯度下降

# 多项式回归

如果你的数据实际上比简单的直线更复杂呢？你依然可以使用线性模型来拟合非线性数据。__一个简单的方法是对每个特征进行加权后作为新的特征，然后训练一个线性模型在这个扩展的特征集。这种方法称为多项式回归。__

## PolynomialFeatures

使用 Scikit-Learning 的 `PolynomialFeatures` 类进行训练数据集的转换，让训练集中每个特征的平方（2 次多项式）作为新特征。

```python
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)
X[ 0 ] # array([ -0.75275929 ])
X_poly[ 0 ] # array([ -0.75275929 ,  0.56664654 ])
```

`X_poly` 现在包含原始特征 $X$ 并加上了这个特征的平方 $X^2$ 。现在可以在这个扩展训练集上使用 `LinearRegression` 模型进行拟合：

```python
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
```

# 过拟合 vs 欠拟合

## 交叉验证

可以使用交叉验证来估计一个模型的泛化能力。如果一个模型在训练集上表现良好，通过交叉验证指标却得出其泛化能力很差，那么你的模型就是过拟合了。如果在这两方面都表现不好，那么它就是欠拟合了。

## 学习曲线

```python
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_errors = [], []

    for m in range(2, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train_predict, y_train[:m]))
        val_errors.append(mean_squared_error(y_val_predict, y_val))

    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")
```

## Pipeline

```python
from sklearn.pipeline import Pipeline

polynomial_regression = Pipeline((
    ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
    ("sgd_reg", LinearRegression()),
))

plot_learning_curves(polynomial_regression, X, y)
```

# 线性模型的正则化

降低模型的过拟合的好方法是正则化这个模型（即限制它）：模型有越少的自由度，就越难以拟合数据。例如，正则化一个多项式模型，一个简单的方法就是减少多项式的阶数。

对于一个线性模型，正则化的典型实现就是约束模型中参数的权重。接下来将介绍三种不同约束权重的方法：__Ridge 回归__，__Lasso 回归__ 和 __Elastic Net__。

## 岭（Ridge）回归

__岭回归（也称为 Tikhonov 正则化）是线性回归的正则化版__：在损失函数上直接加上一个正则项 $ \alpha \sum^n_{i=1}{\theta^2_i} $。这使得学习算法不仅能够拟合数据，而且能够使模型的参数权重尽量的小。

> 注意：这个正则项 __只有在训练过程中才会被加到损失函数。当得到完成训练的模型后，应该使用没有正则化的测量方法去评价模型的表现__。

超参数 $\alpha$ 决定了你想正则化这个模型的强度。如果 $\alpha = 0$ 那此时的岭回归便变为了线性回归。如果 $\alpha$ 非常的大，所有的权重最后都接近于零，最后结果将是一条穿过数据平均值的水平直线。

### 特征缩放

在使用岭回归前，对数据进行放缩（可以使用 `StandardScaler`）是非常重要的，算法对于输入特征的数值尺度（scale）非常敏感。大多数的正则化模型都是这样的。

### 实例

下面是如何使用 Scikit-Learn 来进行正规方程法的求解（使用 Cholesky 法进行矩阵分解）:

```python
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1 , solver="cholesky")
ridge_reg.fit(X, y)
ridge_reg.predict([[ 1.5 ]])
# array([[  1.55071465 ]]
```

使用随机梯度法进行求解：

```python
sgd_reg = SGDRegressor(penalty="l2")
sgd_reg.fit(X, y.ravel())
sgd_reg.predict([[ 1.5 ]])
# array([[  1.13500145 ]])
```

`penalty` 参数指的是正则项的惩罚类型。指定“l2”表明你要在损失函数上添加一项：权重向量 $l_2$ 范数平方的一半，这就是简单的岭回归。

## Lasso 回归

Lasso 回归（也称 Least Absolute Shrinkage，或者 Selection Operator Regression）是另一种正则化版的线性回归：__就像岭回归那样，它也在损失函数上添加了一个正则化项，但是它使用权重向量的 $l_1$ 范数而不是权重向量 $l_2$ 范数平方的一半。__

Lasso 回归的一个重要特征是它 __倾向于完全消除最不重要的特征的权重（即将它们设置为零）__。

下面是一个使用 Scikit-Learn 的 `Lasso` 类的小例子。你也可以使用 `SGDRegressor(penalty="l1")` 来代替它。

```python
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)
lasso_reg.predict([[ 1.5 ]])
# array([  1.53788174 ]
```

## 弹性网络（ElasticNet）

弹性网络介于 Ridge 回归和 Lasso 回归之间。它的正则项是 Ridge 回归和 Lasso 回归正则项的简单混合，同时你可以控制它们的混合率 $r$，当 $r=0$ 时，弹性网络就是 Ridge 回归，当 $r=1$ 时，其就是 Lasso 回归。

该如何选择线性回归，岭回归，Lasso 回归，弹性网络呢？一般来说有一点正则项的表现更好，因此通常你应该避免使用简单的线性回归。岭回归是一个很好的首选项，但是如果你的特征仅有少数是真正有用的，你应该选择 Lasso 和弹性网络。就像我们讨论的那样，它两能够将无用特征的权重降为零。一般来说，弹性网络的表现要比 Lasso 好，因为当特征数量比样本的数量大的时候，或者特征之间有很强的相关性时，Lasso 可能会表现的不规律。

下面是一个使用 Scikit-Learn `ElasticNet`（`l1_ratio` 指的就是混合率 $r$）的简单样本：

```python
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(X, y)
elastic_net.predict([[ 1.5 ]])
# array([ 1.54333232 ])
```

## 早期停止法（Early Stopping）

对于迭代学习算法，有一种非常特殊的正则化方法，就像梯度下降在验证错误达到最小值时立即停止训练那样。我们称为早期停止法。使用批量梯度下降来训练一个非常复杂的模型（一个高阶多项式回归模型）。一般而言 __随着训练的进行，算法一直学习，它在训练集上的预测误差（RMSE）自然而然的下降。然而一段时间后，验证误差停止下降，并开始上升。这意味着模型在训练集上开始出现过拟合。一旦验证错误达到最小值，便提早停止训练__。这种简单有效的正则化方法被 Geoffrey Hinton 称为“完美的免费午餐”

# 逻辑回归

## 决策边界

## Softmax 回归

Logistic 回归模型可以直接推广到支持多类别分类，不必组合和训练多个二分类器，其称为 Softmax 回归或多类别 Logistic 回归。

使用 Softmax 回归对三种鸢尾花进行分类。当使用 `LogisticRregression` 对模型进行训练时，Scikit Learn 默认使用的是一对多模型，但是可以设置 `multi_class` 参数为 `multinomial` 来把它改变为 Softmax 回归。

还必须指定一个支持 Softmax 回归的求解器，例如`lbfgs`求解器。其默认使用 $l_1$ 正则化，你可以使用超参数 $C$ 控制它。

```python
X = iris["data"][:, (2, 3)]  # petal length, petal width 
y = iris["target"]

softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10)
softmax_reg.fit(X, y)
```

> 超参数C的含义和吴恩达老师讲SVM时相同。